In [77]:
from datetime import datetime
import pandas as pd
import json

In [78]:
# depth of this file in the project
file_depth = '../..'

In [79]:
with open(file_depth + '/config/data_1m_config.json') as config_file:
    config = json.load(config_file)

In [80]:
dataset = config['dataset_path']
ratings = pd.read_csv(file_depth + dataset + 'original/ratings.dat', sep='\::', names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')

#### Context from rating timestamp

In [81]:
context = pd.DataFrame()
context['rating_id'] = ratings.index
context['timestamp'] = ratings['timestamp']

In [82]:
context

,rating_id,timestamp
0,0,978300760
1,1,978302109
2,2,978301968
3,3,978300275
4,4,978824291
...,...,...
1000204,1000204,956716541
1000205,1000205,956704887
1000206,1000206,956704746
1000207,1000207,956715648


In [83]:
# create datetime object from timestamp
context['date'] = context['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

In [84]:
# create a week day column from timestamp

"""
0: Monday
1: Tuesday
2: Wednesday
3: Thursday
4: Friday
5: Saturday
6: Sunday
"""
context['day'] = context['timestamp'].apply(lambda x: datetime.fromtimestamp(x).isoweekday())

In [85]:
# create a is_weekday column from day column

"""
0: false / weekend
1: true  / weekday
"""
def get_is_weekday(day):
    if day == 6 or day == 7:
        return 0
    else:
        return 1
    
context['is_weekday'] = context['day'].apply(get_is_weekday)

In [86]:
"""
1: Spring	
2: Summer
3: Fall
4: Winter
"""
def get_season(month):
    if month >= 3 and month <= 5:
        return 1
    elif month >= 6 and month <= 8:
        return 2
    elif month >= 9 and month <= 11:
        return 3
    else:
        return 4
    
context['season'] = context['date'].apply(lambda x: get_season(x.month))

In [87]:
"""
1 - Morning
2 - Afternoon
3 - Evening
4 - Night
"""
def get_part_of_day(hour):
    if hour >= 5 and hour < 12:
        return 1
    elif hour >= 12 and hour < 17:
        return 2
    elif hour >= 17 and hour < 21:
        return 3
    else:
        return 4
    
context['part_of_day'] = context['date'].apply(lambda x: get_part_of_day(x.hour))

In [88]:
with open('holidays.json', 'r') as json_file:
    holiday_dates = json.load(json_file)

def is_date_in_interval(date, start_date, end_date, holiday_name):
    if holiday_name == 'new_years': 
        if date.month == 12:
            if date.day >= start_date.day:
                return True

        elif date.month == 1:
            if date.day <= end_date.day:
                return True
        return False
    else:
        if date.month >= start_date.month and date.month <= end_date.month:
            if date.day >= start_date.day and date.day <= end_date.day:
                return True
        return False


def find_holiday(date):
    for holiday_name, interval in holiday_dates.items():
        start_date = datetime.strptime(interval['start'], '%m-%d')
        end_date = datetime.strptime(interval['end'], '%m-%d')
        if is_date_in_interval(date, start_date, end_date, holiday_name):
            return holiday_name
    return 'no_holiday'

In [89]:
context['holiday'] = context['date'].apply(find_holiday)

In [90]:
holiday_counts = context['holiday'].value_counts()
holiday_counts

holiday
no_holiday        600457
summer_holiday    286263
thanksgiving       82293
christmas          22805
new_years           4727
veterans_day        2747
valentines           917
Name: count, dtype: int64

In [91]:
context = context.drop(columns=['date'])
context = context.drop(columns=['timestamp'])

In [92]:
context

,rating_id,day,is_weekday,season,part_of_day,holiday
0,0,7,0,4,4,new_years
1,1,7,0,4,4,new_years
2,2,7,0,4,4,new_years
3,3,7,0,4,4,new_years
4,4,7,0,4,4,no_holiday
...,...,...,...,...,...,...
1000204,1000204,3,1,1,4,no_holiday
1000205,1000205,3,1,1,4,no_holiday
1000206,1000206,3,1,1,4,no_holiday
1000207,1000207,3,1,1,4,no_holiday


#### Context from user

In [93]:
users = pd.read_csv(file_depth + dataset + 'original/users.dat', sep='\::', names=['user_id', 'gender', 'age', 'occupation', 'zip_code'], engine='python')
users

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [94]:
context['user_id'] = ratings['user_id']

In [95]:
print('are there any null values in gender column: ' + str(users['gender'].isnull().values.any()))
print('unique genders are: ' + ", ".join(users['gender'].unique()))
context = context.merge(users[['user_id', 'gender']], on='user_id', how='left')
context.rename(columns={'gender': 'user_gender'}, inplace=True)

are there any null values in gender column: False
unique genders are: F, M


In [96]:
print('are there any null values in age column: ' + str(users['age'].isnull().values.any()))
print('unique age groups are: ' + ", ".join(map(str, users['age'].unique())))
context = context.merge(users[['user_id', 'age']], on='user_id', how='left')
context.rename(columns={'age': 'user_age'}, inplace=True)

are there any null values in age column: False
unique age groups are: 1, 56, 25, 45, 50, 35, 18


In [97]:
print('are there any null values in occupation column: ' + str(users['occupation'].isnull().values.any()))
print('unique occupation groups are: ' + ", ".join(map(str, users['occupation'].unique())))
context = context.merge(users[['user_id', 'occupation']], on='user_id', how='left')
context.rename(columns={'occupation': 'user_occupation'}, inplace=True)

are there any null values in occupation column: False
unique occupation groups are: 10, 16, 15, 7, 20, 9, 1, 12, 17, 0, 3, 14, 4, 11, 8, 19, 2, 18, 5, 13, 6


In [98]:
def valid_numerical_values(column, column_name):
    is_digit_check = column.str.isdigit()

    message = 'all values of the ' + column_name + ' column are digits'
    for n in is_digit_check:
        if n == False:
            message = 'there are non-digit values in the ' + column_name + ' column'
            break

    print(message)

def non_null_values(column, column_name):
    null_check = column.isnull().values.any()

    if null_check:
        print('there are null values in the ' + column_name + ' column')
    else:
        print('there are no null values in the ' + column_name + ' column')

In [99]:
# Check if there are any null values in zip_code column
non_null_values(users['zip_code'], 'zip_code')

# Check if 'zip_code' consists only of digits
valid_numerical_values(users['zip_code'], 'zip_code')

there are no null values in the zip_code column
there are non-digit values in the zip_code column


There values in zip_code column which are not valid zip codes. Lets see what are those values.

In [100]:
is_digit_check = users['zip_code'].str.isdigit()

non_digit_values = users[~is_digit_check]['zip_code'].value_counts()
if not non_digit_values.empty:
    print("Non-digit values and their occurrences:")
    print(non_digit_values)

Non-digit values and their occurrences:
zip_code
96707-1321    2
98107-2117    1
91790-1161    1
62263-3004    1
01904-1355    1
             ..
64081-8102    1
80004-4448    1
50265-2850    1
08854-3115    1
78705-5221    1
Name: count, Length: 65, dtype: int64


It looks like, that it invalid values have in common zip_code'-xxxx' pattern. Let's keep the first 5 characters of the zip_code and delete the rest.

In [101]:
users['zip_code'] = users['zip_code'].str[:5]

# Check again if there are any null values in zip_code column
non_null_values(users['zip_code'], 'zip_code')

there are no null values in the zip_code column


In [102]:
context = context.merge(users[['user_id', 'zip_code']], on='user_id', how='left')
context.rename(columns={'zip_code': 'user_zip_code'}, inplace=True)

In [103]:
context = context.drop(columns=['user_id'])

In [104]:
context

,rating_id,day,is_weekday,season,part_of_day,holiday,user_gender,user_age,user_occupation,user_zip_code
0,0,7,0,4,4,new_years,F,1,10,48067
1,1,7,0,4,4,new_years,F,1,10,48067
2,2,7,0,4,4,new_years,F,1,10,48067
3,3,7,0,4,4,new_years,F,1,10,48067
4,4,7,0,4,4,no_holiday,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,1000204,3,1,1,4,no_holiday,M,25,6,11106
1000205,1000205,3,1,1,4,no_holiday,M,25,6,11106
1000206,1000206,3,1,1,4,no_holiday,M,25,6,11106
1000207,1000207,3,1,1,4,no_holiday,M,25,6,11106


#### Movies

In [105]:
movies = pd.read_csv(file_depth + dataset + 'original/movies.dat', sep='\::', names=['movie_id', 'title', 'genres'], engine='python', encoding="Latin-1")

# Change genre Children's to ForChildren
movies['genres'] = movies['genres'].str.replace("Children's", "ForChildren")

# Extract year from movie title and return it as context
movies['movie_year'] = movies['title'].str.extract(r'\((\d{4})\)')

print('Does any movie have no year informatrion? : ' + str(movies['movie_year'].isnull().values.any()))
print('Unique years: ')
print(movies['movie_year'].unique())

Does any movie have no year informatrion? : False
Unique years: 
['1995' '1994' '1996' '1976' '1993' '1992' '1988' '1967' '1964' '1977'
 '1965' '1982' '1962' '1990' '1991' '1989' '1937' '1940' '1969' '1981'
 '1973' '1970' '1960' '1955' '1956' '1959' '1968' '1980' '1975' '1986'
 '1948' '1943' '1963' '1950' '1946' '1987' '1997' '1974' '1958' '1949'
 '1972' '1998' '1933' '1952' '1951' '1957' '1961' '1954' '1934' '1944'
 '1942' '1941' '1953' '1939' '1947' '1945' '1938' '1935' '1936' '1926'
 '1932' '1930' '1971' '1979' '1966' '1978' '1985' '1983' '1984' '1931'
 '1922' '1927' '1929' '1928' '1925' '1923' '1999' '1919' '2000' '1920'
 '1921']


#### One-hot encode movie genres

In [106]:
# One-hot encode movie genres
genres = movies['genres'].str.lower().str.get_dummies('|')
genres = genres.add_prefix('genre_')
genres

,genre_action,genre_adventure,genre_animation,genre_comedy,genre_crime,genre_documentary,genre_drama,genre_fantasy,genre_film-noir,genre_forchildren,genre_horror,genre_musical,genre_mystery,genre_romance,genre_sci-fi,genre_thriller,genre_war,genre_western
0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3880,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3881,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [107]:
movies = pd.concat([movies, genres], axis=1)
movies

,movie_id,title,genres,movie_year,genre_action,genre_adventure,genre_animation,genre_comedy,genre_crime,genre_documentary,...,genre_film-noir,genre_forchildren,genre_horror,genre_musical,genre_mystery,genre_romance,genre_sci-fi,genre_thriller,genre_war,genre_western
0,1,Toy Story (1995),Animation|ForChildren|Comedy,1995,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|ForChildren|Fantasy,1995,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,1995,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,2000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),Drama,2000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland (2000),Drama,2000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House (2000),Drama,2000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Create prediction target (ratings) file

In [108]:
ratings_target = pd.DataFrame()
ratings_target['rating_id'] = ratings.index
ratings_target['rating'] = ratings['rating']
ratings_target

,rating_id,rating
0,0,5
1,1,3
2,2,3
3,3,4
4,4,5
...,...,...
1000204,1000204,1
1000205,1000205,5
1000206,1000206,5
1000207,1000207,4


#### Create context data

In [109]:
context_data = ratings.drop(columns=['rating', 'timestamp'])
context_data['rating_id'] = ratings.index
context_data = context_data.merge(context, on='rating_id', how='left')
context_data = context_data.merge(movies, on='movie_id', how='left')
context_data = context_data.drop(columns=['rating_id', 'genres', 'title'])
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,holiday,user_gender,user_age,user_occupation,...,genre_film-noir,genre_forchildren,genre_horror,genre_musical,genre_mystery,genre_romance,genre_sci-fi,genre_thriller,genre_war,genre_western
0,1,1193,7,0,4,4,new_years,F,1,10,...,0,0,0,0,0,0,0,0,0,0
1,1,661,7,0,4,4,new_years,F,1,10,...,0,1,0,1,0,0,0,0,0,0
2,1,914,7,0,4,4,new_years,F,1,10,...,0,0,0,1,0,1,0,0,0,0
3,1,3408,7,0,4,4,new_years,F,1,10,...,0,0,0,0,0,0,0,0,0,0
4,1,2355,7,0,4,4,no_holiday,F,1,10,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,3,1,1,4,no_holiday,M,25,6,...,0,0,0,0,0,0,0,0,0,0
1000205,6040,1094,3,1,1,4,no_holiday,M,25,6,...,0,0,0,0,0,1,0,0,1,0
1000206,6040,562,3,1,1,4,no_holiday,M,25,6,...,0,0,0,0,0,0,0,0,0,0
1000207,6040,1096,3,1,1,4,no_holiday,M,25,6,...,0,0,0,0,0,0,0,0,0,0


In [110]:
# Drop one-hot encoded genres columns from movies 
columns_to_drop = genres.columns
movies = movies.drop(columns=columns_to_drop)

-----------------

#### Save dataframes to csv files

In [111]:
new_context_path = config['new_context_path']

users.to_csv(file_depth + new_context_path + 'users.csv', index=False)
movies.to_csv(file_depth + new_context_path + 'movies.csv', index=False)
ratings.to_csv(file_depth + new_context_path + 'ratings.csv', index=False)
# context.to_csv(file_depth + new_context_path+  'context.csv', index=False)
context_data.to_csv(file_depth + new_context_path + 'context_data.csv', index=False)
ratings_target.to_csv(file_depth + new_context_path + 'ratings_target.csv', index=False)